conda_tensorflow_p36

In [1]:
import subprocess, json

In [2]:
def add_class(method):
    return "public class Test {\n"+method+"\n}"

In [3]:
def remove_class(parse_tree):
    return {'classbody': parse_tree['compilationUnit'][0]['typeDeclaration'][1]['classDeclaration'][2]['classBody'][1:-1]}

In [4]:
def parse(code):
    code = add_class(code)
    tree = subprocess.check_output(['java', '-jar', 'mavenproject1-1.0-SNAPSHOT-jar-with-dependencies.jar', code])
    tree = json.loads(tree)
    tree = remove_class(tree)
    return tree

In [18]:
import ast
def tree2code(tree, indentation):
    code = ""
    parent = list(tree.keys())[0]
    children = tree[parent]
    if 'type' in tree:
        code = tree['text'] + ' '
        if tree['text'] in [';', '{', '}']:
            code += '\n'
            if tree['text'] == '{':
                indentation += 1
            elif tree['text'] == '}':
                indentation -= 1
            code += '\t'*indentation
        #return code, indentation
    elif len(children)==1 and parent == "literal":
        child = children[0]
        child_text = child["text"]
        #type(ast.literal_eval(child_text))
        try:
            if child_text in ['null', 'true', 'false']:
                code = child_text + " "
            elif type(ast.literal_eval(child_text)) == type("STRING"):
                code = "STRING00 "
            elif type(ast.literal_eval(child_text)) == type(1):
                code = "INT00 "
            elif type(ast.literal_eval(child_text)) == type(1.1):
                code = "FLOAT00 "
        except:
            code = "NUM00 "
    else:
        root = list(tree.keys())[0]
        children = tree[root]
        for child in children:
            t, indentation = tree2code(child, indentation)
            code += t
    return code, indentation

In [32]:
import json
import copy
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data

def dump_dict(data, name):
    with open(name, 'w', encoding="utf8") as f:
        json.dump(data, f)

In [17]:
## Special Tokens
# STRING
# _NUM

In [13]:
code = """
private int findPLV ( int M_PriceList_ID ) { Timestamp priceDate = null ; String dateStr = Env . getContext ( Env . getCtx ( ) , p_WindowNo , STRING ) ; if ( dateStr != null && dateStr . length ( ) > _NUM ) priceDate = Env . getContextAsDate ( Env . getCtx ( ) , p_WindowNo , STRING ) ; else { dateStr = Env . getContext ( Env . getCtx ( ) , p_WindowNo , STRING ) ; if ( dateStr != null && dateStr . length ( ) > _NUM ) priceDate = Env . getContextAsDate ( Env . getCtx ( ) , p_WindowNo , STRING ) ; } if ( priceDate == null ) priceDate = new Timestamp ( System . currentTimeMillis ( ) ) ; log . config ( STRING + M_PriceList_ID + STRING + priceDate ) ; int retValue = _NUM ; String sql = STRING + STRING + STRING + STRING + STRING + STRING ; try { PreparedStatement pstmt = DB . prepareStatement ( sql , null ) ; pstmt . setInt ( _NUM , M_PriceList_ID ) ; ResultSet rs = pstmt . executeQuery ( ) ; while ( rs . next ( ) && retValue == _NUM ) { Timestamp plDate = rs . getTimestamp ( _NUM ) ; if ( ! priceDate . before ( plDate ) ) retValue = rs . getInt ( _NUM ) ; } rs . close ( ) ; pstmt . close ( ) ; } catch ( SQLException e ) { log . log ( Level . SEVERE , sql , e ) ; } Env . setContext ( Env . getCtx ( ) , p_WindowNo , STRING , retValue ) ; return retValue ; }
"""
tree = parse(code)
tree

{'classbody': [{'classBodyDeclaration': [{'modifier': [{'classOrInterfaceModifier': [{'type': 33,
         'text': 'private'}]}]},
    {'memberDeclaration': [{'methodDeclaration': [{'typeSpec': [{'primitiveType': [{'type': 27,
             'text': 'int'}]}]},
        {'type': 100, 'text': 'findPLV'},
        {'formalParameters': [{'type': 57, 'text': '('},
          {'formalParameterList': [{'formalParameter': [{'typeSpec': [{'primitiveType': [{'type': 27,
                   'text': 'int'}]}]},
              {'variableDeclaratorId': [{'type': 100,
                 'text': 'M_PriceList_ID'}]}]}]},
          {'type': 58, 'text': ')'}]},
        {'methodBody': [{'block': [{'type': 59, 'text': '{'},
            {'blockStatement': [{'localVariableDeclarationStatement': [{'localVariableDeclaration': [{'typeSpec': [{'classOrInterfaceType': [{'type': 100,
                       'text': 'Timestamp'}]}]},
                  {'variableDeclarators': [{'variableDeclarator': [{'variableDeclaratorId':

In [14]:
code = tree2code(tree,-1)[0]
print(code)

private int findPLV ( int M_PriceList_ID ) { 
Timestamp priceDate = null ; 
String dateStr = Env . getContext ( Env . getCtx ( ) , p_WindowNo , STRING ) ; 
if ( dateStr != null && dateStr . length ( ) > _NUM ) priceDate = Env . getContextAsDate ( Env . getCtx ( ) , p_WindowNo , STRING ) ; 
else { 
	dateStr = Env . getContext ( Env . getCtx ( ) , p_WindowNo , STRING ) ; 
	if ( dateStr != null && dateStr . length ( ) > _NUM ) priceDate = Env . getContextAsDate ( Env . getCtx ( ) , p_WindowNo , STRING ) ; 
	} 
if ( priceDate == null ) priceDate = new Timestamp ( System . currentTimeMillis ( ) ) ; 
log . config ( STRING + M_PriceList_ID + STRING + priceDate ) ; 
int retValue = _NUM ; 
String sql = STRING + STRING + STRING + STRING + STRING + STRING ; 
try { 
	PreparedStatement pstmt = DB . prepareStatement ( sql , null ) ; 
	pstmt . setInt ( _NUM , M_PriceList_ID ) ; 
	ResultSet rs = pstmt . executeQuery ( ) ; 
	while ( rs . next ( ) && retValue == _NUM ) { 
		Timestamp plDate = rs . getTi

In [ ]:
# add class
# parse
# build dictionary
# Build Numto compress
# compress
# tokenize
# remove class tokens

# Building Parse Tree

In [33]:
# Use glob to find all files
# Or just list the 6 directories manually
paths = [
    'code_summarization/dev/code.original',
    'code_summarization/test/code.original',
    'code_summarization/train/code.original'
]
paths

['code_summarization/dev/code.original',
 'code_summarization/test/code.original',
 'code_summarization/train/code.original']

In [36]:
import datetime

#path = 'code_summarization/train/code.original'
for path in paths:
    start = datetime.datetime.now()
    print("=====[", path,"]=====")
    i = 0
    error_count = 0
    parse_trees = []
    codes = []
    with open(path) as data:
        for raw_code in data:
            try:
                end = datetime.datetime.now()
                if i%500 == 0: print("[", i, "]", end-start)
                raw_code = raw_code.replace("STRING", "\"STRING\"").replace("_NUM", "1")
                tree = parse(raw_code)
                code = tree2code(tree, -1)[0]
            except Exception as e:
                print(e)
                print("Line:",i,"is corrupted. Cannot build dictionary.")
                error_count += 1
                tree = "{}"
                code = ""
            parse_trees.append(tree)
            codes.append({"code": code})
            i+=1

    dump_dict(parse_trees, path.replace("original", "tree"))
    dump_dict(codes, path.replace("original", "noliteral"))
    end = datetime.datetime.now()
    print("Dictionary building complete in - ",end-start)
    print("Error Count", error_count)

=====[ code_summarization/dev/code.original ]=====
[ 0 ] 0:00:01.312469
[ 500 ] 0:02:41.982281
[ 1000 ] 0:22:05.500365
[ 1500 ] 0:24:43.864442
[ 2000 ] 0:27:27.896460
[ 2500 ] 0:30:01.367123
[ 3000 ] 0:32:34.534325
[ 3500 ] 0:35:05.583873
[ 4000 ] 0:37:38.314485
[ 4500 ] 0:40:23.299780
[ 5000 ] 0:43:02.799612
[ 5500 ] 0:45:48.744744
[ 6000 ] 0:48:26.169399
[ 6500 ] 0:51:10.574182
[ 7000 ] 0:54:10.393760
[ 7500 ] 0:57:05.023494
[ 8000 ] 1:00:00.804039
[ 8500 ] 1:02:43.917919
Dictionary building complete in -  1:04:14.831206
Error Count 0
=====[ code_summarization/test/code.original ]=====
[ 0 ] 1:04:16.304373
[ 500 ] 1:06:47.155864
[ 1000 ] 1:09:41.348292
[ 1500 ] 1:12:33.855576
[ 2000 ] 1:15:19.036959
[ 2500 ] 1:18:03.088125
[ 3000 ] 1:20:49.363653
[ 3500 ] 1:23:34.984649
[ 4000 ] 1:26:15.692592
[ 4500 ] 1:28:57.781763
[ 5000 ] 1:31:40.595355
[ 5500 ] 1:34:15.282945
[ 6000 ] 1:36:46.284227
[ 6500 ] 1:39:17.940677
[ 7000 ] 1:41:51.258433
[ 7500 ] 1:44:22.820012
[ 8000 ] 1:46:53.144747
[

In [37]:
codes[0:10]

[{'code': '@ Override public int runCommand ( boolean mergeErrorIntoOutput , String ... commands ) throws IOException , InterruptedException { \nreturn runCommand ( mergeErrorIntoOutput , new ArrayList < String > ( Arrays . asList ( commands ) ) ) ; \n} \n'},
 {'code': 'private int findPLV ( int M_PriceList_ID ) { \nTimestamp priceDate = null ; \nString dateStr = Env . getContext ( Env . getCtx ( ) , p_WindowNo , STRING00 ) ; \nif ( dateStr != null && dateStr . length ( ) > INT00 ) priceDate = Env . getContextAsDate ( Env . getCtx ( ) , p_WindowNo , STRING00 ) ; \nelse { \n\tdateStr = Env . getContext ( Env . getCtx ( ) , p_WindowNo , STRING00 ) ; \n\tif ( dateStr != null && dateStr . length ( ) > INT00 ) priceDate = Env . getContextAsDate ( Env . getCtx ( ) , p_WindowNo , STRING00 ) ; \n\t} \nif ( priceDate == null ) priceDate = new Timestamp ( System . currentTimeMillis ( ) ) ; \nlog . config ( STRING00 + M_PriceList_ID + STRING00 + priceDate ) ; \nint retValue = INT00 ; \nString sql

In [ ]:
## build freq_dict


In [ ]:
## add func_names from abstract_code_dict, set count: 0

for i in range in len(abstract_code_dict):
    func = '$F'+str(i)
    freq_dict[func] = 0

In [ ]:
## initialize vocab_size
vocab_size = 50000

In [ ]:
import re

num_to_compress = 0
for i in range(len(abstract_code_keys)):
    func_name = abstract_code_dict[abstract_code_keys[i]]['compressed_name']
    code = abstract_code_keys[i]
    count = abstract_code_dict[abstract_code_keys[i]]['count']
    most_frequent_list = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
    
    if count >= freq_dict[most_frequent_list[vocab_size]]:
        token_list = [x for x in code.split() if x[:3] != '$id']
        for token in token_list:
            freq_dict[token] -= count
        freq_dict[func_name] += count
    else:
        num_to_compress = i
        print("Number to Compress:",i)
        break
most_frequent_list = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)